<a href="https://colab.research.google.com/github/eljanmahammadli/Kaggle/blob/master/llm-detect-ai-generated-text/notebooks/bert_fine_tuning_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00


In [1]:
import os, sys, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import Dataset
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
nltk.download('stopwords')

Mounted at /content/drive


In [ ]:
# read data depending on whether it is on kaggle, colab or local
if "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
    print("Running on Kaggle!")
    test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
    submission = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')
    org_train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
    extra = pd.read_csv("/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv", sep=',')

elif "google.colab" in sys.modules:
    print("Running on Google Colab!")
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = '/content/drive/MyDrive/Kaggle/LLM_Detect_AI_Generated_Text/data/'
    test = pd.read_csv(data_path + "test_essays.csv")
    submission = pd.read_csv(data_path + "sample_submission.csv")
    org_train = pd.read_csv(data_path + "train_essays.csv")
    extra = pd.read_csv(data_path + "train_v2_drcat_02.csv", sep=",")
else:
    print("Running locally.")
    test = pd.read_csv("./data/test_essays.csv")
    submission = pd.read_csv("./data/sample_submission.csv")
    org_train = pd.read_csv("./data/train_essays.csv")
    train = pd.read_csv("./data/train_v2_drcat_02.csv", sep=",")

In [8]:
# drop duplicates
extra = extra.drop_duplicates(subset=['text'])
extra.reset_index(drop=True, inplace=True)

# Text Preprocessing
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuations
    words = text.split()  # Tokenize
    words = [word.lower() for word in words if word.isalpha()]  # Lowercase and remove non-alphabetic words
    words = [word for word in words if word not in stop_words]  # Remove stop words
    return ' '.join(words)

extra['clean_text'] = extra['text'].apply(clean_text)

In [9]:
# split into train, test, and val set
extra = extra.sample(frac=1.0, random_state=42).reset_index(drop=True)
train, val = train_test_split(extra, test_size=0.2, random_state=42, stratify=extra['label'])
X_train, X_val, y_train, y_val = train_test_split(extra['clean_text'], extra['label'], test_size=0.2, random_state=42)

In [10]:
# Tokenization and Encoding for BERT
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased', do_lower_case=True, padding=True, truncation=True, max_length=128
)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [14]:
train_dataset = Dataset.from_dict({"text": X_train.tolist()})
val_dataset = Dataset.from_dict({"text": X_val.tolist()})

# Tokenize the datasets in parallel
def tokenize_batch(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, return_tensors="pt")

train_dataset = train_dataset.map(tokenize_batch, batched=True)
val_dataset = val_dataset.map(tokenize_batch, batched=True)

# Access the tokenized data
encoded_train = train_dataset["input_ids"], train_dataset["attention_mask"]
encoded_val = val_dataset["input_ids"], val_dataset["attention_mask"]


Map:   0%|          | 0/35894 [00:00<?, ? examples/s]

Map:   0%|          | 0/8974 [00:00<?, ? examples/s]

In [15]:
# Convert labels to tensors
train_labels = torch.tensor(y_train.values)
val_labels = torch.tensor(y_val.values)

In [44]:
# Create TensorDatasets
train_dataset = TensorDataset(torch.tensor(encoded_train[0]), torch.tensor(encoded_train[1]), torch.tensor(train_labels))
val_dataset = TensorDataset(torch.tensor(encoded_val[0]), torch.tensor(encoded_val[1]), torch.tensor(val_labels))


<ipython-input-44-8526433ac0dd>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(encoded_train[0]), torch.tensor(encoded_train[1]), torch.tensor(train_labels))
<ipython-input-44-8526433ac0dd>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(encoded_val[0]), torch.tensor(encoded_val[1]), torch.tensor(val_labels))


In [45]:
# DataLoader for efficient processing
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [46]:
# Define the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [48]:
# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=3e-4, correct_bias=False)
epochs = 10

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping to avoid exploding gradients
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_train_loss:.2f}")

In [ ]:
# Validation loop
model.eval()
val_preds = []
val_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        val_preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

In [ ]:
# Calculate validation accuracy
val_accuracy = accuracy_score(val_labels, val_preds)
print(f"Validation Accuracy: {val_accuracy:.2f}")

In [ ]:
import time
while True:
  time.sleep(1)

In [ ]:
# Test data processing
test_inputs = tokenizer(test_essays['text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Move input tensor to the same device as the model
test_inputs = {key: value.to(device) for key, value in test_inputs.items()}

# Generate predictions using your trained model
with torch.no_grad():
    outputs = model(**test_inputs)
    logits = outputs.logits

# Assuming the first column of logits corresponds to the negative class (non-AI-generated)
# and the second column corresponds to the positive class (AI-generated)
predictions = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()  # Move predictions back to CPU

# Create a submission DataFrame with essay IDs and corresponding predictions
submission = pd.DataFrame({
    'id': test_essays['id'],
    'generated': predictions
})

# Save the submission DataFrame to a CSV file
submission.to_csv('/kaggle/working/submission.csv', index=False)